In [16]:
import os
import shutil

from google.cloud.storage import Client, transfer_manager

# add service account key as default credentials
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './possible-cocoa-283823-363154d3490a.json'

scratch = os.path.join(os.path.expanduser('~'), "scratch")

dest_dir = os.path.join(scratch, "quickdraw")
download_dir = os.path.join(scratch, "download-quickdraw")

In [17]:
def download_bucket_with_transfer_manager(bucket_name, destination_directory="", workers=8):

    storage_client = Client()
    bucket = storage_client.bucket(bucket_name)

    blob_names = [blob.name for blob in bucket.list_blobs() if "full/raw" in blob.name]
    
    print(f"Found {len(blob_names)} categories")
    
    results = transfer_manager.download_many_to_path(
        bucket, blob_names, destination_directory=destination_directory, max_workers=workers
    )

    for name, result in zip(blob_names, results):
        # The results list is either `None` or an exception for each blob in
        # the input list, in order.

        if isinstance(result, Exception):
            print("Failed to download {} due to exception: {}".format(name, result))

In [18]:
# takes a while but only have to do once
download_bucket_with_transfer_manager("quickdraw_dataset", download_dir)

Found 345 categories


In [22]:
# move files to more convinient location
os.makedirs(dest_dir)

downloaded_files = os.listdir(os.path.join(download_dir, "full/raw"))

for file in downloaded_files:
  full_file_path = os.path.join(download_dir, "full/raw", file)
  dest_full_path = os.path.join(dest_dir, file)
  shutil.move(full_file_path, dest_full_path)

shutil.rmtree(download_dir)